<a href="https://colab.research.google.com/github/NOTTEI/Datasets/blob/master/pop2piano_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
  # Copyright [2022] [pop2piano]
  #      Licensed under the Apache License, Version 2.0 (the "License");
  #      you may not use this file except in compliance with the License.
  #      You may obtain a copy of the License at
  #         http://www.apache.org/licenses/LICENSE-2.0
  #      Unless required by applicable law or agreed to in writing, software
  #      distributed under the License is distributed on an "AS IS" BASIS,
  #      WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
  #      See the License for the specific language governing permissions and
  #      limitations under the License.

#@title Setup Environment
#@markdown - Install Pop2Piano and its dependencies (may take a few minutes).
#@markdown - We highly recommend to use a GPU runtime.
#@markdown - click: Runtime >> Change Runtime Type >> GPU

!apt-get install -y fluidsynth
!git clone https://github.com/sweetcocoa/pop2piano/
!cd pop2piano
!pip install pretty-midi==0.2.9 omegaconf==2.1.1 youtube-dl==2021.12.17 transformers==4.16.1 pytorch-lightning essentia==2.1b6.dev609 note-seq==0.0.3 pyFluidSynth==1.3.0
!wget https://github.com/sweetcocoa/pop2piano/releases/download/dpi_2k_epoch/model-1999-val_0.67311615.ckpt

In [ ]:
#@title Import codes

import os 
import sys
sys.path.append("pop2piano")
import glob
import random

import torch
import torchaudio
import librosa
import numpy as np
import pandas as pd
import IPython.display as ipd
import soundfile as sf
from google.colab import files

from tqdm.auto import tqdm
from omegaconf import OmegaConf
import note_seq

from utils.dsp import get_stereo
from utils.demo import download_youtube
from transformer_wrapper import TransformerWrapper
from midi_tokenizer import MidiTokenizer, extrapolate_beat_times
from preprocess.beat_quantizer import extract_rhythm, interpolate_beat_times

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
config = OmegaConf.load("pop2piano/config.yaml")
wrapper = TransformerWrapper(config)
wrapper = wrapper.load_from_checkpoint("model-1999-val_0.67311615.ckpt", config=config).to(device)
model = "dpipqxiy"
wrapper.eval()
""

In [ ]:
#@title Select Arranger(Composer)

composer = "composer1" #@param['composer1', 'composer2', 'composer3', 'composer4', 'composer5', 'composer6', 'composer7', 'composer8', 'composer9', 'composer10', 'composer11', 'composer12', 'composer13', 'composer14', 'composer15', 'composer16', 'composer17', 'composer18', 'composer19', 'composer20', 'composer21']

In [ ]:
#@title Upload audio file (Supports : An audio file like wav, mp3)
file = files.upload()
audio_file = list(file.keys())[0]

In [ ]:
#@title Result

pm, composer, mix_path, midi_path = wrapper.generate(
    audio_path=audio_file, 
    composer=composer, 
    model=model,
    show_plot=True, 
    save_midi=True, 
    save_mix=True, 
)
note_seq.plot_sequence(note_seq.midi_to_note_sequence(pm))

In [ ]:
#@title Download generated Piano cover MIDI
files.download(midi_path)